# Triangular matrices - part 2

Consider the two triangular matrices presented below:

$$\mathbf{U} 
= \left[ \begin{array}{ccccc}
u_{11} & u_{12} & u_{13} & \cdots & u_{1N} \\
       & u_{22} & u_{23} & \cdots & u_{2N} \\
       &        & u_{33} & \cdots & u_{3N}  \\
       &        &        & \ddots & \vdots  \\
       &        &        &        & u_{NN}
\end{array} \right]
$$

and

$$\mathbf{L} 
= \left[ \begin{array}{ccccc}
l_{11} & & & & \\
\vdots & \ddots & & & \\
l_{N-2 \, 1} & \cdots & l_{N-2\,N-2} & &\\
l_{N-1 \, 1} & \cdots & l_{N-1\,N-2} & l_{N-1\,N-1} &\\
l_{N1} & \cdots & l_{N\,N-2} & l_{N\,N-1} & l_{N\,N}
\end{array} \right] \: .
$$

As we can see, a large fraction of the elements forming these matrices is zero. For the case in which $N$ is large, we need to think about **efficient storage schemes**. For example (see the notebook [`diagonal_matrices.ipynb`](https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/diagonal_matrices.ipynb)), a diagonal matrix $\mathbf{D}$ can be efficiently stored as a vector $\mathbf{d}$. Is this particular case, there is an easy relationship between the elements of $\mathbf{d}$ and the elements of the original diagonal matrix $\mathbf{D}$. It is also easy to rewrite the algorithms for computing the product of $\mathbf{D}$ and a full matrix or vector by using the vector $\mathbf{d}$.

In the case of triangular matrices, the storage schemes are more complicated. Let's create the triangular matrices presented above and use them to illustrate different storage schemes. The following cells use the routines [`numpy.random.rand`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html), [`numpy.triu`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.triu.html), [`numpy.tril`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tril.html), and [`numpy.around`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.around.html).

In [1]:
import numpy as np

In [2]:
N = 4
A = np.around(100*np.random.rand(N,N), decimals=3)
U = np.triu(A)
L = np.tril(A)

In [5]:
print A, '\n'
print U, '\n'
print L

[[ 97.74   76.983  92.359  88.527]
 [ 76.681  10.941   3.158  98.564]
 [ 46.761  48.537  47.158  44.024]
 [ 83.151  57.731   2.521  90.861]] 

[[ 97.74   76.983  92.359  88.527]
 [  0.     10.941   3.158  98.564]
 [  0.      0.     47.158  44.024]
 [  0.      0.      0.     90.861]] 

[[ 97.74    0.      0.      0.   ]
 [ 76.681  10.941   0.      0.   ]
 [ 46.761  48.537  47.158   0.   ]
 [ 83.151  57.731   2.521  90.861]]


**1) column-based storage scheme for $\mathbf{U}$**

This scheme stores the triangular matrix $\mathbf{U}$, column by column, in a vector $\mathbf{u}_{c}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 2 & 3 \\
0 & 4 & 5 \\
0 & 0 & 6
\end{bmatrix}}_{\mathbf{U}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 4 \\ 3 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{u}_{c}}
$$

An element $ij$ of the matrix $\mathbf{U}$ is associated with the $k$th element of the vector $\mathbf{u}_{c}$, where 

$$
k = i - 1 + j*(j - 1)/2
$$

In [10]:
for j in range(1,N+1):
    for i in range(1,j+1):
        k = i - 1 + j*(j - 1)/2
        print '%d%d %d' % (i, j, k)

11 0
12 1
22 2
13 3
23 4
33 5
14 6
24 7
34 8
44 9


**2) row-based storage scheme for $\mathbf{U}$**

This scheme stores the triangular matrix $\mathbf{U}$, row by row, in a vector $\mathbf{u}_{r}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 2 & 3 \\
0 & 4 & 5 \\
0 & 0 & 6
\end{bmatrix}}_{\mathbf{U}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{u}_{r}}
$$

An element $ij$ of the matrix $\mathbf{U}$ is associated with the $k$th element of the vector $\mathbf{u}_{r}$, where 

$$
k = j - 1 + (i - 1)*(2*n - i)/2
$$

In [11]:
for i in range(1,N+1):
    for j in range(i,N+1):
        k = j - 1 + (i - 1)*(2*N - i)/2
        print '%d%d %d' % (i, j, k)

11 0
12 1
13 2
14 3
22 4
23 5
24 6
33 7
34 8
44 9


**3) column-based storage scheme for $\mathbf{L}$**

This scheme stores the triangular matrix $\mathbf{L}$, column by column, in a vector $\mathbf{l}_{c}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 0 & 0 \\
2 & 3 & 0 \\
4 & 5 & 6
\end{bmatrix}}_{\mathbf{L}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 4 \\ 3 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{l}_{c}}
$$

An element $ij$ of the matrix $\mathbf{L}$ is associated with the $k$th element of the vector $\mathbf{l}_{c}$, where 

$$
k = i - 1 + (j - 1)*(2*n - j)/2
$$

In [15]:
for j in range(1,N+1):
    for i in range(j,N+1):
        k = i - 1 + (j - 1)*(2*N - j)/2
        print '%d%d %d' % (i, j, k)

11 0
21 1
31 2
41 3
22 4
32 5
42 6
33 7
43 8
44 9


**4) row-based storage scheme for $\mathbf{L}$**

This scheme stores the triangular matrix $\mathbf{L}$, row by row, in a vector $\mathbf{l}_{r}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 0 & 0 \\
2 & 3 & 0 \\
4 & 5 & 6
\end{bmatrix}}_{\mathbf{L}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{l}_{r}}
$$

An element $ij$ of the matrix $\mathbf{L}$ is associated with the $k$th element of the vector $\mathbf{l}_{r}$, where 

$$
k = j - 1 + i*(i - 1)/2
$$

In [13]:
for i in range(1,N+1):
    for j in range(1,i+1):
        k = j - 1 + i*(i - 1)/2
        print '%d%d %d' % (i, j, k)

11 0
21 1
22 2
31 3
32 4
33 5
41 6
42 7
43 8
44 9


### References

* Golub, G. H. and Van Loan, C. F. Matrix computations, 4th edition, Johns Hopkins University Press, 2013

* [IBM Knowledge Center - Engineering and Scientific Subroutine Library](https://www.ibm.com/support/knowledgecenter/en/SSFHY8_5.5.0/com.ibm.cluster.essl.v5r5.essl100.doc/am5gr_data.htm)

* [Intel developer zone - Matrix Storage Schemes for LAPACK Routines](https://software.intel.com/en-us/mkl-developer-reference-c-matrix-storage-schemes-for-lapack-routines)